In [1]:
!pip install scipy==1.0.0
!pip install numpy==1.17.4


     |████████████████████████████████| 50.0MB 90kB/s 
ERROR: tensorflow 2.2.0rc3 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.0.0 which is incompatible.
ERROR: seaborn 0.10.0 has requirement scipy>=1.0.1, but you'll have scipy 1.0.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: cvxpy 1.0.31 has requirement scipy>=1.1.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 20.0MB 238kB/s 
ERROR: tensorflow 2.2.0rc3 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.0.0 which is incompatible.
ERROR: seaborn 0.10.0 has requirement scipy>=1.0.1, but you'll have scipy 1.0.0 which

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
# Import libraries
import time
import keras
from keras import Input
from keras.layers import BatchNormalization, Activation, Add, LeakyReLU, Dense, MaxPooling2D
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.callbacks import TensorBoard
from keras.models import Model
from keras.optimizers import Adam
import glob
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from scipy.misc import imresize, imread
from keras import backend as K
import math
import numpy
from skimage.measure import compare_ssim
import csv
import os.path

In [0]:
def build_vgg():
    """
    Builds a pre-trained VGG19 model that outputs image features extracted at the
    third block of the model
    """
    input_shape = (256, 256, 3)
    vgg = VGG19(weights="imagenet")
    # Set the outputs to outputs of last conv. layer in block 3
    # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
    vgg.outputs = [vgg.layers[9].output]

    img = Input(shape=input_shape)

    # Extract the image features
    img_features = vgg(img)

    return Model(inputs=[img], outputs=[img_features], name='VGG')

In [0]:
# Residual block for generator network
def residual_block(x):
    """
    Residual block
    """
    filters = [64, 64]
    kernel_size = 3
    strides = 1
    padding = "same"
    momentum = 0.8
    activation = PReLU()

    res = Conv2D(filters=filters[0], kernel_size=kernel_size,
                 strides=strides, padding=padding)(x)
    res = Activation(activation=activation)(res)
    res = BatchNormalization(momentum=momentum)(res)

    res = Conv2D(filters=filters[1], kernel_size=kernel_size,
                 strides=strides, padding=padding)(res)
    res = BatchNormalization(momentum=momentum)(res)

    # Add res and x
    res = Add()([res, x])
    return res

In [0]:
def build_generator():
    """
    Create a generator network using the hyperparameter values defined below
    :return:
    """
    residual_blocks = 16
    momentum = 0.8
    input_shape = (64, 64, 3)
    counter = 0

    # Input Layer of the generator network
    input_layer = Input(shape=input_shape)

    # Add the pre-residual block
    gen1 = Conv2D(filters=64, kernel_size=9, strides=1, padding='same')(input_layer)
    gen1 = Activation(PReLU())(gen1)
    # Add 16 residual blocks
    res = residual_block(gen1)
    for i in range(residual_blocks - 1):
        res = residual_block(res)

    # Add the post-residual block
    gen2 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(res)
    gen2 = BatchNormalization(momentum=momentum)(gen2)

    # Take the sum of the output from the pre-residual block(gen1) and
    #  the post-residual block(gen2)
    gen3 = Add(name='trial')([gen2, gen1])

    # Add an upsampling block
    gen4 = UpSampling2D(size=2)(gen3)
    # gen4 = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(gen3)
    gen4 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen4)
    gen4 = Activation(PReLU())(gen4)

    # Add another upsampling block
    gen5 = UpSampling2D(size=2)(gen4)
    # gen5 = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(gen4)
    gen5 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen5)
    gen5 = Activation(PReLU())(gen5)

    # Output convolution layer
    gen6 = Conv2D(filters=3, kernel_size=9, strides=1, padding='same', activation='tanh')(gen5)

    # Auto-Encoder
    x = Conv2D(512, (3, 3), activation='relu', strides=1, padding='same')(gen6)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # at this point the representation is (7, 7, 32)

    x = Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', strides=1, padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='tanh', strides=1, padding='same')(x)
    # Keras model
    model = Model(inputs=[input_layer], outputs=[decoded],
                  name='generator')

    return model

In [0]:
# Discriminator Network
def build_discriminator():
    """
    Create a discriminator network using the hyperparameter values defined below
    :return:
    """
    leakyrelu_alpha = 0.2
    momentum = 0.8
    input_shape = (256, 256, 3)

    input_layer = Input(shape=input_shape)

    # Add the first convolution block
    dis1 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
    dis1 = LeakyReLU(alpha=leakyrelu_alpha)(dis1)

    # Add the 2nd convolution block
    dis2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(dis1)
    dis2 = LeakyReLU(alpha=leakyrelu_alpha)(dis2)
    dis2 = BatchNormalization(momentum=momentum)(dis2)

    # Add the third convolution block
    dis3 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(dis2)
    dis3 = LeakyReLU(alpha=leakyrelu_alpha)(dis3)
    dis3 = BatchNormalization(momentum=momentum)(dis3)

    # Add the fourth convolution block
    dis4 = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(dis3)
    dis4 = LeakyReLU(alpha=leakyrelu_alpha)(dis4)
    dis4 = BatchNormalization(momentum=0.8)(dis4)

    # Add the fifth convolution block
    dis5 = Conv2D(256, kernel_size=3, strides=1, padding='same')(dis4)
    dis5 = LeakyReLU(alpha=leakyrelu_alpha)(dis5)
    dis5 = BatchNormalization(momentum=momentum)(dis5)

    # Add the sixth convolution block
    dis6 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same')(dis5)
    dis6 = LeakyReLU(alpha=leakyrelu_alpha)(dis6)
    dis6 = BatchNormalization(momentum=momentum)(dis6)

    # Add the seventh convolution block
    dis7 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(dis6)
    dis7 = LeakyReLU(alpha=leakyrelu_alpha)(dis7)
    dis7 = BatchNormalization(momentum=momentum)(dis7)

    # Add the eight convolution block
    dis8 = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(dis7)
    dis8 = LeakyReLU(alpha=leakyrelu_alpha)(dis8)
    dis8 = BatchNormalization(momentum=momentum)(dis8)

    # Add a dense layer
    dis9 = Dense(units=1024)(dis8)
    dis9 = LeakyReLU(alpha=0.2)(dis9)

    # Last dense layer - for classification
    output_gen = Dense(units=1, activation='sigmoid')(dis9)

    model = Model(inputs=[input_layer], outputs=[output_gen], name='discriminator')
    return model

In [0]:
def sample_images(data_dir, batch_size, high_resolution_shape, low_resolution_shape):

    # Make a list of all images inside the data directory
    all_images = data_dir

    # Choose a random batch of images
    images_batch = np.random.choice(all_images, size=batch_size)

    low_resolution_images = []
    high_resolution_images = []

    for img in images_batch:
        # Get an ndarray of the current image
        img1 = imread(img, mode='RGB')
        img1 = img1.astype(np.float32)

        # Resize the image
        img1_high_resolution = imresize(img1, high_resolution_shape)
        img1_low_resolution = imresize(img1, low_resolution_shape)

        # Do a random horizontal flip
        if np.random.random() < 0.5:
          img1_high_resolution = np.fliplr(img1_high_resolution)
          img1_low_resolution = np.fliplr(img1_low_resolution)

        high_resolution_images.append(img1_high_resolution)
        low_resolution_images.append(img1_low_resolution)

    # Convert the lists to Numpy NDArrays
    return np.array(high_resolution_images), np.array(low_resolution_images)

In [0]:
def save_images(low_resolution_images, high_resolution_images, generated_images, path):
    fig = plt.figure()
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(low_resolution_images[0])
    ax.axis("off")
    ax.set_title("Low Resolution")

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(high_resolution_images[0])
    ax.axis("off")
    ax.set_title("Original")

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(generated_images[0])
    ax.axis("off")
    ax.set_title("Generated")

    plt.savefig(path)
    plt.clf()

In [0]:
def PSNR(true_image, predicted_image):
    mse = numpy.mean((true_image - predicted_image) ** 2)
    Pixel_max = 1.0
    return 20 * math.log10(Pixel_max / math.sqrt(mse))

In [0]:
if __name__ == '__main__':
    # Define hyperparameters
    data_dir = glob('./Dataset/*')
    epochs = 10001
    batch_size = 2
    lr = 0.0002
    mode = 'train'
    # Shape of low-resolution and high-resolution images
    low_resolution_shape = (64, 64, 3)
    high_resolution_shape = (256, 256, 3)

    # Common optimizer for all networks
    common_optimizer = Adam(lr, 0.9)

    # Training the model
    if mode == 'train':

        # Building and compiling the networks
        vgg = build_vgg()
        vgg.trainable = False
        # print("VGG")
        # print(vgg.summary())
        vgg.compile(optimizer=common_optimizer, loss='mse', metrics=['accuracy'])

        discriminator = build_discriminator()
        # print("discrminator")
        # print(discriminator.summary())
        discriminator.compile(optimizer=common_optimizer, loss='mse', metrics=['accuracy'])
        print(discriminator.metrics_names)

        generator = build_generator()
        print("generator")
        print(generator.summary())

        # Building and compiling the adversarial network
        # High and Low resolution inputs to the network
        input_high_resolution = Input(shape=high_resolution_shape)
        input_low_resolution = Input(shape=low_resolution_shape)

        # Generating high resolution images from the generator
        generated_high_resolution_images = generator(input_low_resolution)

        # Extracting high resolution features using VGG network
        features = vgg(generated_high_resolution_images)

        # Discriminator model is turned off during adversarial training
        discriminator.trainable = False
        discriminator.compile(optimizer=common_optimizer, loss='mse', metrics=['accuracy'])

        # Probability of generated high resolution images
        probs = discriminator(generated_high_resolution_images)

        # Creating the adversarial model
        adversarial_model = Model([input_low_resolution, input_high_resolution], [probs, features])
        # print("Adversarial")
        # print(adversarial_model.summary())
        adversarial_model.compile(optimizer=common_optimizer, loss=['binary_crossentropy', 'mse'],
                                  loss_weights=[1e-3, 1])
        # print(adversarial_model.metrics_names)

        # Add Tensorboard
        # tensorboard = TensorBoard(log_dir="logs_imagenet_res/".format(time.time()))
        # tensorboard.set_model(generator)
        # tensorboard.set_model(discriminator)

        # Training
        for epoch in range(epochs):
            print("Epoch :{}".format(epoch))
            # experiment.log_parameter('epoch', epoch)

            # Training the discriminator network

            high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                          high_resolution_shape=high_resolution_shape,
                                                                          low_resolution_shape=low_resolution_shape)
            high_resolution_images = high_resolution_images / 255.0
            low_resolution_images = low_resolution_images / 255.0

            generated_high_resolution_images = generator.predict(low_resolution_images)

            # Generating batch of real and fake labels
            real_labels = np.ones((batch_size, 16, 16, 1))
            fake_labels = np.zeros((batch_size, 16, 16, 1))

            d_loss_real = discriminator.train_on_batch(high_resolution_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_high_resolution_images, fake_labels)

            # write_log(tensorboard, 'd_loss_real', d_loss_real[0], epoch)
            # write_log(tensorboard, 'd_loss_real_acc', d_loss_real[1], epoch)
            # write_log(tensorboard, 'd_loss_fake', d_loss_fake[0], epoch)
            # write_log(tensorboard, 'd_loss_fake_acc', d_loss_fake[1], epoch)

            # Data logging to csv
            with open('d_loss_real', 'a', newline='') as myfile:
                fileEmpty = os.stat('d_loss_real').st_size == 0
            
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                headers = ['Loss', 'Acc']
                writer = csv.DictWriter(myfile, fieldnames=headers)
            
                if fileEmpty:
                    writer.writeheader()
            
                wr.writerow(d_loss_real)
            
            with open('d_loss_fake', 'a', newline='') as myfile:
                fileEmpty = os.stat('d_loss_fake').st_size == 0
            
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                headers = ['Loss', 'Acc']
                writer = csv.DictWriter(myfile, fieldnames=headers)
            
                if fileEmpty:
                    writer.writeheader()
            
                wr.writerow(d_loss_fake)

            # Calculating the discriminator loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            print("d_loss :", d_loss)
            # print(type(d_loss))

            # Data logging to csv
            with open('d_loss', 'a', newline='') as myfile:
                fileEmpty = os.stat('d_loss').st_size == 0
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                headers = ['Loss', 'Acc']
                writer = csv.DictWriter(myfile, fieldnames=headers)
            
                if fileEmpty:
                    writer.writeheader()
                wr.writerow(d_loss)

            # Training the generator network
            high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                          high_resolution_shape=high_resolution_shape,
                                                                          low_resolution_shape=low_resolution_shape)

            high_resolution_images = high_resolution_images / 255.0
            low_resolution_images = low_resolution_images / 255.0

            real_labels = np.ones((batch_size, 16, 16, 1))

            image_features = vgg.predict(high_resolution_images)

            g_loss = adversarial_model.train_on_batch([low_resolution_images, high_resolution_images],
                                                      [real_labels, image_features])
            print("g_loss :", g_loss)
            # print(type(g_loss))

            # Data logging to csv
            with open('g_loss', 'a', newline='') as myfile:
                fileEmpty = os.stat('g_loss').st_size == 0
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                headers = ['Loss', 'Discriminator_loss', 'vgg_loss']
                writer = csv.DictWriter(myfile, fieldnames=headers)
            
                if fileEmpty:
                    writer.writeheader()
                wr.writerow(g_loss)

            # Write the losses to Tensorboard
            # write_log(tensorboard, 'g_loss', g_loss[0], epoch)
            # write_log(tensorboard, 'discriminator_loss', g_loss[1], epoch)
            # write_log(tensorboard, 'ResNet_loss', g_loss[2], epoch)

            # write_log(tensorboard, 'd_loss', d_loss[0], epoch)
            # write_log(tensorboard, 'd_acc', d_loss[1], epoch)

            # Saving images
            if epoch % 100 == 0:
                high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                              high_resolution_shape=high_resolution_shape,
                                                                              low_resolution_shape=low_resolution_shape)
                # Normalizing the images
                high_resolution_images = high_resolution_images / 255.0
                low_resolution_images = low_resolution_images / 255.0

                generated_images = generator.predict_on_batch(low_resolution_images)

                # low_resolution_images = 0.5 * low_resolution_images + 0.5
                # generated_images = 0.5 * generated_images + 0.5
                # high_resolution_images = 0.5 * high_resolution_images + 0.5

                for index, img in enumerate(generated_images):
                    save_images(low_resolution_images, high_resolution_images, generated_images,
                                path="Results/img_{}".format(epoch))

                # Calculating PSNR and SSIM metrics
                psnr = PSNR(high_resolution_images[0], generated_images[0])
                print("PSNR: {}".format(psnr))
                psnr = [float(psnr)]
                # with open('PSNR', 'a', newline='') as myfileg:
                #     wr = csv.writer(myfileg, quoting=csv.QUOTE_ALL)
                #     wr.writerow(psnr)

                (score, diff) = compare_ssim(high_resolution_images[0], generated_images[0], full=True,
                                             multichannel=True)
                print("SSIM: {}".format(score))
                score = [float(score)]
                # with open('SSIM', 'a', newline='') as myfileg:
                #     wr = csv.writer(myfileg, quoting=csv.QUOTE_ALL)
                #     wr.writerow(score)

        # generator.save_weights("generator_imagenet_res.h5")
        # discriminator.save_weights("discriminator_imagenet_res.h5")

        # Predict Results
    if mode == 'Predict':
        # Build discriminator and generator
        discriminator = build_discriminator()
        generator = build_generator()

        # Load weights from training
        discriminator.load_weights("discriminator_imagenet_res.h5")
        generator.load_weights("generator_imagenet_res.h5")

        # Load test images
        data_dir = glob('./Predict/*')
        high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=2,
                                                                      high_resolution_shape=high_resolution_shape,
                                                                      low_resolution_shape=low_resolution_shape)
        # Normalizing the image
        high_resolution_images = high_resolution_images / 255.0
        low_resolution_images = low_resolution_images / 255.0

        generated_images = generator.predict_on_batch(low_resolution_images)

        # low_resolution_images = 0.5 * low_resolution_images + 0.5
        # generated_images = 0.5 * generated_images + 0.5
        # high_resolution_images = 0.5 * high_resolution_images + 0.5

        # Calculating PSNR and SSIM of images
        psnr1 = PSNR(high_resolution_images[0], generated_images[0])
        (score1, diff1) = compare_ssim(high_resolution_images[0], generated_images[0], full=True, multichannel=True)
        print("PSNR_first_image: {}".format(psnr1))
        print("SSIM_first_image: {}".format(score1))

        psnr2 = PSNR(high_resolution_images[1], generated_images[1])
        (score2, diff2) = compare_ssim(high_resolution_images[1], generated_images[1], full=True, multichannel=True)
        print("PSNR_second_image: {}".format(psnr2))
        print("SSIM_second_image: {}".format(score2))

        # Saving images
        for index, img in enumerate(generated_images):
            save_images(low_resolution_images, high_resolution_images, generated_images,
                        path="results/gen_{}".format(index))


['loss', 'accuracy']


/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


generator
Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_55 (Conv2D)              (None, 64, 64, 64)   15616       input_8[0][0]                    
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 64, 64, 64)   262144      conv2d_55[0][0]                  
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 64, 64, 64)   36928       activation_20[0][0]              
________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.



d_loss : [0.2839967 0.5097656]
g_loss : [33.698208, 1.1369637, 33.69707]


...

PSNR: 15.452149073317102
SSIM: 0.7896198625121347
Epoch :1101
d_loss : [0.29293925 0.49804688]
g_loss : [1.589331, 1.1280004, 1.5882031]
Epoch :1102
d_loss : [0.2940668 0.5029297]
g_loss : [2.1145618, 1.1447604, 2.1134171]
Epoch :1103
d_loss : [0.29177988 0.5097656 ]
g_loss : [1.7507527, 1.1348585, 1.7496178]
Epoch :1104
d_loss : [0.29018173 0.48828125]
g_loss : [2.23974, 1.134332, 2.2386055]
Epoch :1105
d_loss : [0.29050088 0.5       ]
g_loss : [2.480174, 1.118119, 2.479056]
Epoch :1106
d_loss : [0.2932305  0.49023438]
g_loss : [1.6874151, 1.1397935, 1.6862754]
Epoch :1107
d_loss : [0.2917726 0.4951172]
g_loss : [1.6608037, 1.1254425, 1.6596782]
Epoch :1108
d_loss : [0.29475352 0.4892578 ]
g_loss : [1.0086968, 1.1417459, 1.007555]
Epoch :1109
d_loss : [0.2930708 0.5029297]
g_loss : [2.1923223, 1.1430331, 2.1911793]
Epoch :1110
d_loss : [0.29114735 0.49804688]
g_loss : [2.1513507, 1.1319702, 2.1502187]
Epoch :1111
d_loss : [0.2910031 0.4970703]
g_loss : [1.6268263, 1.1307842, 1.6256955

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


PSNR: 21.015185954751942
SSIM: 0.7526720400059365
Epoch :1201
d_loss : [0.2910673 0.4873047]
g_loss : [1.1178671, 1.1368859, 1.1167302]
Epoch :1202
d_loss : [0.29282337 0.48828125]
g_loss : [1.4934412, 1.1279709, 1.4923133]
Epoch :1203
d_loss : [0.29008675 0.5029297 ]
g_loss : [1.6824548, 1.1332109, 1.6813216]
Epoch :1204
d_loss : [0.28844577 0.5058594 ]
g_loss : [1.162765, 1.1508377, 1.1616142]
Epoch :1205
d_loss : [0.29084826 0.5107422 ]
g_loss : [1.0745245, 1.1295081, 1.073395]
Epoch :1206
d_loss : [0.292863 0.5     ]
g_loss : [1.7044319, 1.1469572, 1.703285]
Epoch :1207
d_loss : [0.29383373 0.50097656]
g_loss : [0.713322, 1.1389976, 0.712183]
Epoch :1208
d_loss : [0.2884085 0.5019531]
g_loss : [1.5489874, 1.1206033, 1.5478668]
Epoch :1209
d_loss : [0.2938778  0.49609375]
g_loss : [1.8008764, 1.1044346, 1.7997719]
Epoch :1210
d_loss : [0.28711572 0.50097656]
g_loss : [1.5563099, 1.1494303, 1.5551605]
Epoch :1211
d_loss : [0.2900844  0.50097656]
g_loss : [1.5228001, 1.1056192, 1.5216

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


PSNR: 23.959163385816886
SSIM: 0.8342429422850387
Epoch :1301
d_loss : [0.2912364  0.51660156]
g_loss : [0.7412559, 1.1416776, 0.7401142]
Epoch :1302
d_loss : [0.29392183 0.5019531 ]
g_loss : [1.4100561, 1.1424459, 1.4089136]
Epoch :1303
d_loss : [0.29240286 0.5126953 ]
g_loss : [0.6824068, 1.137397, 0.6812694]
Epoch :1304
d_loss : [0.28898704 0.515625  ]
g_loss : [1.170627, 1.1149182, 1.169512]
Epoch :1305
d_loss : [0.2892226  0.50878906]
g_loss : [1.134789, 1.1179018, 1.133671]
Epoch :1306
d_loss : [0.29502842 0.4951172 ]
g_loss : [1.2942497, 1.1285712, 1.2931211]
Epoch :1307
d_loss : [0.29256815 0.49414062]
g_loss : [0.9706725, 1.119249, 0.96955323]
Epoch :1308
d_loss : [0.29159963 0.4970703 ]
g_loss : [1.8830054, 1.1303012, 1.881875]
Epoch :1309
d_loss : [0.29116553 0.49121094]
g_loss : [1.0562005, 1.1209085, 1.0550796]
Epoch :1310
d_loss : [0.29353765 0.50097656]
g_loss : [0.9501758, 1.109869, 0.9490659]
Epoch :1311
d_loss : [0.29171285 0.49609375]
g_loss : [0.9582318, 1.1552501, 